In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

In [5]:
train['lgb_pred'] = np.load("../input/tps-aug-2021-lgb/lgb_oof.npy")
test['lgb_pred'] = np.load("../input/tps-aug-2021-lgb/lgb_pred.npy")

train['xgb_pred'] = np.load("../input/tps-aug-2021-xgb/xgb_oof.npy")
test['xgb_pred'] = np.load("../input/tps-aug-2021-xgb/xgb_pred.npy")

train['xgb2_pred'] = np.load("../input/tps-aug-2021-xgb-pseudo2-pred/3xgb_oof.npy")+np.load("../input/tps-aug-2021-xgb-pseudo2-pred/4xgb_oof.npy")
test['xgb2_pred'] = np.load("../input/tps-aug-2021-xgb-pseudo2-pred/3xgb_pred.npy")+np.load("../input/tps-aug-2021-xgb-pseudo2-pred/4xgb_pred.npy")

train['ctb2_pred'] = np.load("../input/tps-aug-2021-catb-pseudo2/ctb_oof.npy")
test['ctb2_pred'] = np.load("../input/tps-aug-2021-catb-pseudo2/ctb_pred.npy")


# Preprocessing

In [6]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [7]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [8]:
pseudo = pd.read_csv("../input/blending-tool-tps-aug-2021/blend.csv/0.part")[target]
test_pseudo = pd.concat([test, pseudo], axis=1)
all_pseudo = pd.concat([train, test_pseudo]).reset_index(drop=True)

In [9]:
useful_features = scale_features + ['lgb_pred', 'xgb_pred','xgb2_pred','ctb2_pred']

# Optuna

In [10]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [11]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo[useful_features], y=all_pseudo[target]):
  """
  """
  param_space = {
                    'objective': 'regression',
              'metric':'rmse',
               #'device':'gpu',  # Use GPU acceleration
               # 'gpu_platform_id': 0,
                #    'gpu_device_id': 0,
               'n_estimators':N_ESTIMATORS,
              'learning_rate':trial.suggest_uniform('learning_rate', 1e-3, 1e-1),
               'subsample': trial.suggest_uniform('subsample', 0.2, 1.0),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.2, 1.0),
                'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
             #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 0.01, 50),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 0.01, 50),
            'min_child_weight':trial.suggest_uniform('min_child_weight', 100, 500),
                 'min_child_samples':trial.suggest_int('min_child_samples', 5, 250),
          'importance_type': 'gain'
                }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo[useful_features].iloc[trn_idx], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_rn(X_train)

  X_valid, y_valid = all_pseudo[useful_features].iloc[oof_idx], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]

  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )


    lgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, lgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [12]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-28 03:26:43,010] A new study created in memory with name: no-name-5f6df95a-e3d4-474e-aa39-47bf4fa5e5e5


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[58]	valid_0's rmse: 7.74759
Training until validation scores don't improve for 200 rounds


[I 2021-08-28 03:27:16,298] Trial 0 finished with value: 7.746530502556741 and parameters: {'learning_rate': 0.04744603349824434, 'subsample': 0.2835255174317608, 'colsample_bytree': 0.9352143587168733, 'subsample_freq': 5, 'reg_lambda': 13.47054249042285, 'reg_alpha': 47.719384953009964, 'min_child_weight': 232.5421357887502, 'min_child_samples': 243}. Best is trial 0 with value: 7.746530502556741.


Early stopping, best iteration is:
[51]	valid_0's rmse: 7.74727
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[38]	valid_0's rmse: 7.74527
Training until validation scores don't improve for 200 rounds


[I 2021-08-28 03:27:57,789] Trial 1 finished with value: 7.744976969410505 and parameters: {'learning_rate': 0.08308531538923612, 'subsample': 0.7873530816742165, 'colsample_bytree': 0.899101218212899, 'subsample_freq': 2, 'reg_lambda': 48.30266168546053, 'reg_alpha': 14.520496808032021, 'min_child_weight': 188.07114783827754, 'min_child_samples': 192}. Best is trial 1 with value: 7.744976969410505.


...

Early stopping, best iteration is:
[48]	valid_0's rmse: 7.74509
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[44]	valid_0's rmse: 7.747
Training until validation scores don't improve for 200 rounds


[I 2021-08-28 03:46:11,793] Trial 28 finished with value: 7.745961340353425 and parameters: {'learning_rate': 0.0589788477446095, 'subsample': 0.6319674745709629, 'colsample_bytree': 0.6810352173954924, 'subsample_freq': 4, 'reg_lambda': 38.4640752337413, 'reg_alpha': 41.39511900236889, 'min_child_weight': 260.94401483344757, 'min_child_samples': 223}. Best is trial 25 with value: 7.7443511015243764.


Early stopping, best iteration is:
[46]	valid_0's rmse: 7.74563
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[58]	valid_0's rmse: 7.74629
Training until validation scores don't improve for 200 rounds


[I 2021-08-28 03:46:47,678] Trial 29 finished with value: 7.745309158485609 and parameters: {'learning_rate': 0.04756202181585209, 'subsample': 0.7650909083770525, 'colsample_bytree': 0.586204978685408, 'subsample_freq': 7, 'reg_lambda': 15.914689238099674, 'reg_alpha': 32.96590102044819, 'min_child_weight': 237.05384911756303, 'min_child_samples': 107}. Best is trial 25 with value: 7.7443511015243764.


Early stopping, best iteration is:
[92]	valid_0's rmse: 7.74526
Number of finished trials: 30
Best trial: {'learning_rate': 0.06708778228872987, 'subsample': 0.6682126072656112, 'colsample_bytree': 0.6797661362187238, 'subsample_freq': 4, 'reg_lambda': 29.62573177775194, 'reg_alpha': 8.474459963363486, 'min_child_weight': 109.35156229664032, 'min_child_samples': 143}


In [13]:
study.best_params

{'learning_rate': 0.06708778228872987,
 'subsample': 0.6682126072656112,
 'colsample_bytree': 0.6797661362187238,
 'subsample_freq': 4,
 'reg_lambda': 29.62573177775194,
 'reg_alpha': 8.474459963363486,
 'min_child_weight': 109.35156229664032,
 'min_child_samples': 143}

# Log

===== 2 preds ======

7.744188593192304 no noise ver1 final

7.7462672672725175 row-wise ver3

7.74692198293704 random ver4

====== 4 preds ======